## Packages

In [1]:
import json
import pandas as pd
import os
import sys
import requests

print(sys.executable)
print(os.getcwd())



C:\Users\Guilherme\.conda\envs\nlp2\python.exe
C:\Users\Guilherme\Documents\Programming\Python\DataScience\TCC_stock_price\pipeline


## Functions

In [2]:
def getDateOutOfDatetimeToIndex(df):
    df['time'] = df['date'].apply(lambda x: str(x).split()[1])
    df['date'] = df['date'].apply(lambda x: str(x).split()[0])
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    return df


def makeDataset(df1_raw, df2_raw, shift=False, onlyPetr=False):
    df1 = df1_raw.copy()
    df2 = df2_raw.copy()
    # 27/12/2020 10:25 -> 27-12-2020

    df2['profit'] = (df2['Close'] - df2['Open']) / df2['Open']
    df2['profit (%)'] = df2['profit'].apply(lambda x: round(x * 100, 2))
    th = 0.0025
    df2['label'] = df2['profit'].apply(lambda x: 0 if x < -th else 2 if x > th else 1 if -th <= x <= th else None)
    # df2['label'] = df2['profit'].apply(lambda x: 0 if x <=0 else 1 if x > 0 else None)

    # profit will be influenced by news on same day (Shift
    if shift:
        df2['label'] = df2['label'].shift(-1)
        # remove row with nan value
        df2.dropna(inplace=True)

    df = df1.merge(df2, on='date', how='inner')

    df = df.loc[df['time'] <= '17:00:00']

    if onlyPetr:
        df['petr'] = df['tags'].apply(
            lambda x: 1 if ('PETR3' in x) or ('PETR4' in x) or ('Petrobras' in x) else 0)
        df = df[df['petr'] == 1]
        df = df.reset_index()

    return df

## Load data

### News

In [3]:
json_data_path = 'https://raw.githubusercontent.com/guico3lho/DataScience_Assets/main/Datasets/financial_market/suno/suno-petr4.json'

# url = 'https://raw.githubusercontent.com/guico3lho/TCC/main/assets/Datasets/suno/suno-petr4.json?token=GHSAT0AAAAAABYDRQ4ZASOZJOMO2OAKI4MEZAOJOKA'
r = requests.get(json_data_path)
data = r.json()
df_suno_petr_raw = pd.DataFrame(data)
df_suno_petr_raw

,topic,title,date,search_date,url,tags
0,Negócios,Petrobras (PETR4) finaliza venda do Polo Peroá...,03/08/2022 13:46,2022-08-04 00:21:19,https://www.suno.com.br/noticias/petrobras-pet...,"[Petrobras (PETR4), 3R Petroleum (RRRP3), Petr..."
1,Economia,Conselheiro da Petrobras (PETR4) sugere congel...,17/06/2022 19:52,2022-08-04 00:21:35,https://www.suno.com.br/noticias/petrobras-pet...,"[Petrobras (PETR4), Arthur Lira, Jair Bolsonar..."
2,Negócios,Radar: MRV (MRVE3) paga dividendos milionários...,20/06/2022 23:05,2022-08-04 00:21:35,https://www.suno.com.br/noticias/radar-petrobr...,"[Ibovespa, Ibovespa, Magazine Luiza (MGLU3), M..."
3,Política,Líder dos caminhoneiros sobre voucher: “Não pr...,22/06/2022 17:24,2022-08-04 00:21:35,https://www.suno.com.br/noticias/lider-caminho...,"[Petrobras (PETR4), combustíveis, diesel, Jair..."
4,Negócios,Petrobras (PETR4): Petroleiros aprovam indicat...,10/07/2022 09:40,2022-08-04 00:21:35,https://www.suno.com.br/noticias/petrobras-pet...,"[Petrobras (PETR4), greves, Petrobras (PETR4),..."
...,...,...,...,...,...,...
1425,Mercado,"Ibovespa fecha em queda de 0,35%, a 104.109,07...",28/07/2020 17:37,2022-05-03 16:22:55,https://www.suno.com.br/noticias/ibovespa-fech...,"[AES Tietê (TIET3/TIET4/TIET11), BNDES, Iboves..."
1426,Negócios,"Petrobras faz pré-pagamento de US$ 3,5 bi em l...",28/07/2020 15:00,2022-05-03 16:22:56,https://www.suno.com.br/noticias/petrobras-pre...,"[PETR3, Petrobras (PETR4)]"
1427,Negócios,Petrobras (PETR4) divulga teaser de venda de p...,28/07/2020 11:26,2022-05-03 16:22:57,https://www.suno.com.br/noticias/petrobras-pet...,"[PETR3, Petrobras (PETR4)]"
1428,Negócios,Agenda do Dia: Petrobras; Yduqs; Oi; AES Tietê...,28/07/2020 09:49,2022-05-03 16:22:57,https://www.suno.com.br/noticias/agenda-do-dia...,"[AES Tietê (TIET3/TIET4/TIET11), agenda do dia..."


In [4]:
df_suno_petr_format_datetime = df_suno_petr_raw.copy()
df_suno_petr_format_datetime['date'] = pd.to_datetime(df_suno_petr_format_datetime['date'], format='%d/%m/%Y %H:%M')
df_suno_petr_format_datetime

,topic,title,date,search_date,url,tags
0,Negócios,Petrobras (PETR4) finaliza venda do Polo Peroá...,2022-08-03 13:46:00,2022-08-04 00:21:19,https://www.suno.com.br/noticias/petrobras-pet...,"[Petrobras (PETR4), 3R Petroleum (RRRP3), Petr..."
1,Economia,Conselheiro da Petrobras (PETR4) sugere congel...,2022-06-17 19:52:00,2022-08-04 00:21:35,https://www.suno.com.br/noticias/petrobras-pet...,"[Petrobras (PETR4), Arthur Lira, Jair Bolsonar..."
2,Negócios,Radar: MRV (MRVE3) paga dividendos milionários...,2022-06-20 23:05:00,2022-08-04 00:21:35,https://www.suno.com.br/noticias/radar-petrobr...,"[Ibovespa, Ibovespa, Magazine Luiza (MGLU3), M..."
3,Política,Líder dos caminhoneiros sobre voucher: “Não pr...,2022-06-22 17:24:00,2022-08-04 00:21:35,https://www.suno.com.br/noticias/lider-caminho...,"[Petrobras (PETR4), combustíveis, diesel, Jair..."
4,Negócios,Petrobras (PETR4): Petroleiros aprovam indicat...,2022-07-10 09:40:00,2022-08-04 00:21:35,https://www.suno.com.br/noticias/petrobras-pet...,"[Petrobras (PETR4), greves, Petrobras (PETR4),..."
...,...,...,...,...,...,...
1425,Mercado,"Ibovespa fecha em queda de 0,35%, a 104.109,07...",2020-07-28 17:37:00,2022-05-03 16:22:55,https://www.suno.com.br/noticias/ibovespa-fech...,"[AES Tietê (TIET3/TIET4/TIET11), BNDES, Iboves..."
1426,Negócios,"Petrobras faz pré-pagamento de US$ 3,5 bi em l...",2020-07-28 15:00:00,2022-05-03 16:22:56,https://www.suno.com.br/noticias/petrobras-pre...,"[PETR3, Petrobras (PETR4)]"
1427,Negócios,Petrobras (PETR4) divulga teaser de venda de p...,2020-07-28 11:26:00,2022-05-03 16:22:57,https://www.suno.com.br/noticias/petrobras-pet...,"[PETR3, Petrobras (PETR4)]"
1428,Negócios,Agenda do Dia: Petrobras; Yduqs; Oi; AES Tietê...,2020-07-28 09:49:00,2022-05-03 16:22:57,https://www.suno.com.br/noticias/agenda-do-dia...,"[AES Tietê (TIET3/TIET4/TIET11), agenda do dia..."


In [5]:
json_data_path = 'https://raw.githubusercontent.com/guico3lho/DataScience_Assets/main/Datasets/financial_market/moneytimes/moneytimes-petr4.json'

# url = 'https://raw.githubusercontent.com/guico3lho/TCC/main/assets/Datasets/suno/suno-petr4.json?token=GHSAT0AAAAAABYDRQ4ZASOZJOMO2OAKI4MEZAOJOKA'
r = requests.get(json_data_path)
data = r.json()

df_moneytimes_petr_raw = pd.DataFrame(data)

df_moneytimes_petr_format_datetime = df_moneytimes_petr_raw.copy()
# df_moneytimes_petr_format_datetime = getDateOutOfDatetimeToIndex(df_moneytimes_petr_format_datetime)
df_moneytimes_petr_format_datetime['date'] = pd.to_datetime(df_moneytimes_petr_format_datetime['date'],
                                                            format='%d/%m/%Y - %H:%M')
df_moneytimes_petr_format_datetime.rename(columns={'link': 'url'}, inplace=True)
df_moneytimes_petr_format_datetime

,topic,title,date,search_date,url,tags
0,Mercados,Ibovespa dispara hoje por quê? Inflação no Bra...,2022-05-11 12:14:00,2022-05-11 19:03:09,https://www.moneytimes.com.br/ibovespa-dispara...,"[Ações, Alimentos, Ativa Investimentos, B3, Bo..."
1,Combustíveis,"Defasagem do diesel cai para 17% após aumento,...",2022-05-10 13:40:00,2022-05-11 19:03:17,https://www.moneytimes.com.br/defasagem-do-die...,"[Amazonas, Bahia, Combustíveis, Diesel, Estadã..."
2,Carteira Recomendada,"Adeus, Ibovespa: 5 ações do BB Investimentos p...",2022-05-08 17:18:00,2022-05-11 19:03:22,https://www.moneytimes.com.br/adeus-ibovespa-5...,"[Ações, Alpargatas, BB Investimentos, Bradesco..."
3,Política,Setor de etanol não se surpreenderá se após Al...,2022-05-11 11:57:00,2022-05-11 19:03:22,https://www.moneytimes.com.br/setor-de-etanol-...,"[Agro Times, Bento Albuquerque, Etanol, Jair B..."
4,Petróleo,EUA perguntaram se Petrobras poderia aumentar ...,2022-05-10 08:23:00,2022-05-11 19:03:22,https://www.moneytimes.com.br/eua-perguntaram-...,"[Bento Albuquerque, Brasil, Combustíveis, Comm..."
...,...,...,...,...,...,...
6124,George Soros,George Soros volta a comprar ação da Petrobras...,2016-11-23 12:22:00,2022-05-11 21:19:57,https://www.moneytimes.com.br/george-soros-vol...,"[George Soros, Petrobras]"
6125,Comprar ou vender?,"Petrobras deve aumentar gasolina em breve, diz...",2016-12-02 18:32:00,2022-05-11 21:19:57,https://www.moneytimes.com.br/petrobras-deve-a...,"[Comprar ou vender?, Itaú BBA, Opep, Petrobras]"
6126,Comprar ou vender?,XP recoloca Petrobras em carteira recomendada,2016-11-21 19:55:00,2022-05-11 21:20:00,https://www.moneytimes.com.br/xp-recoloca-petr...,"[Comprar ou vender?, Petrobras, XP Investimentos]"
6127,Benndorf,Petrobras: Dia é decisivo para tendência de al...,2016-12-01 11:59:00,2022-05-11 21:20:00,https://www.moneytimes.com.br/petrobras-dia-e-...,"[Benndorf, Bolsa, Comprar ou vender?, Petrobras]"


In [6]:
df_petr4_news = pd.concat([df_suno_petr_format_datetime[['title', 'tags', 'url', 'date']],
                           df_moneytimes_petr_format_datetime[['title', 'tags', 'url', 'date']]])

df_petr4_news.reset_index(inplace=True, drop=True)

df_petr4_news['title'] = df_petr4_news['title'].map(lambda s: s.replace('\xa0', ''))
df_petr4_news = getDateOutOfDatetimeToIndex(df_petr4_news)

df_petr4_news

,title,tags,url,time
date,,,,
2016-09-21,"Petrobras: Menos é mais, assim como nos casame...","[Comprar ou vender?, Petrobras, Petróleo]",https://www.moneytimes.com.br/petrobras-menos-...,00:31:00
2016-11-02,As 5 apostas do BB Investimentos para novembro,"[BB Investimentos, Bolsa, Comprar ou vender?, ...",https://www.moneytimes.com.br/as-5-apostas-do-...,04:53:00
2016-11-02,Projeção nada boa para Petrobras na volta do f...,"[Petrobras, Petróleo]",https://www.moneytimes.com.br/projecao-nada-bo...,15:04:00
2016-11-08,Como a Petrobras pode ganhar com Hillary,"[Donald Trump, Hillary Clinton, Petrobras, Pet...",https://www.moneytimes.com.br/como-a-petrobras...,18:55:00
2016-11-21,Quer comprar ação da Petrobras? Passa lá no po...,"[Comprar ou vender?, Fusões & Aquisições, Petr...",https://www.moneytimes.com.br/quer-comprar-aca...,12:52:00
...,...,...,...,...
2022-07-31,Dividendos bilionários da Petrobras (PETR4) sã...,"[Petrobras (PETR4), Petrobras (PETR4)]",https://www.suno.com.br/noticias/dividendos-bi...,16:28:00
2022-08-01,Radar: Petrobras (PETR4) lidera ranking de div...,"[Petrobras (PETR4), CSN (CSNA3), EDP Brasil (E...",https://www.suno.com.br/noticias/radar-petrobr...,23:21:00
2022-08-02,Carteira de dividendos: BTG troca Vale (VALE3)...,"[Petrobras (PETR4), BTG Pactual (BPAC11/BPAC3/...",https://www.suno.com.br/noticias/dividendos-ca...,12:34:00


### Historical Prices

In [7]:
# import opendatasets as od
# od.download("https://www.kaggle.com/datasets/guico3lho/petr4-historical-data-2016-09-2022-08")

In [8]:
df_petr4_historical_prices = pd.read_csv('../assets/petr4-historical-data-2016-09-2022-08/PETR4_2016_09__2022_08.csv')
df_petr4_historical_prices.rename(columns={'Date': 'date'}, inplace=True)
df_petr4_historical_prices.set_index('date', inplace=True)
df_petr4_historical_prices

,Open,High,Low,Close,Adj Close,Volume
date,,,,,,
2016-09-21,13.600000,13.690000,13.400000,13.660000,5.028104,61094100
2016-09-22,13.890000,14.090000,13.840000,14.000000,5.153255,47574500
2016-09-23,13.980000,13.980000,13.540000,13.690000,5.039146,53264300
2016-09-26,13.590000,13.690000,13.390000,13.400000,4.932401,35545800
2016-09-27,13.400000,13.400000,12.870000,13.120000,4.829334,58098200
...,...,...,...,...,...,...
2022-07-27,31.299999,31.469999,30.620001,31.350000,18.657389,56097200
2022-07-28,31.590000,32.630001,31.430000,32.290001,19.216816,114996500
2022-07-29,33.599998,34.660000,32.830002,34.150002,20.323761,181480600


## Making Dataset

In [14]:
df_raw = makeDataset(df_petr4_news, df_petr4_historical_prices)
df_raw

,title,tags,url,time,Open,High,Low,Close,Adj Close,Volume,profit,profit (%),label
date,,,,,,,,,,,,,
2016-09-21,"Petrobras: Menos é mais, assim como nos casame...","[Comprar ou vender?, Petrobras, Petróleo]",https://www.moneytimes.com.br/petrobras-menos-...,00:31:00,13.600000,13.690000,13.400000,13.660000,5.028104,61094100,0.004412,0.44,2
2016-11-21,Quer comprar ação da Petrobras? Passa lá no po...,"[Comprar ou vender?, Fusões & Aquisições, Petr...",https://www.moneytimes.com.br/quer-comprar-aca...,12:52:00,15.050000,15.580000,14.880000,15.580000,5.734836,59122000,0.035216,3.52,2
2016-11-23,George Soros volta a comprar ação da Petrobras...,"[George Soros, Petrobras]",https://www.moneytimes.com.br/george-soros-vol...,12:22:00,15.720000,15.990000,15.530000,15.850000,5.834219,42490100,0.008270,0.83,2
2016-11-28,Itaú BBA remove ação da Petrobras de carteira ...,"[Ambev, Comprar ou vender?, Itaú BBA, Petrobras]",https://www.moneytimes.com.br/itau-bba-remove-...,15:13:00,15.200000,15.560000,15.200000,15.460000,5.690664,38747500,0.017105,1.71,2
2016-11-28,Vem aí o novo velho Ibovespa,"[Banco do Brasil, BRF, Comprar ou vender?, Emb...",https://www.moneytimes.com.br/vem-ai-o-novo-ve...,08:59:00,15.200000,15.560000,15.200000,15.460000,5.690664,38747500,0.017105,1.71,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-28,"Petrobras (PETR4) pagará dividendos de R$ 6,73...","[Petrobras (PETR4), dividendos, Petrobras (PET...",https://www.suno.com.br/noticias/petrobras-pet...,16:02:00,31.590000,32.630001,31.430000,32.290001,19.216816,114996500,0.022159,2.22,2
2022-07-29,"Ibovespa sobe com Petrobras (PETR4), mas Vale ...","[Ibovespa, 3R Petroleum (RRRP3), americanas S....",https://www.suno.com.br/noticias/ibovespa-hoje...,10:42:00,33.599998,34.660000,32.830002,34.150002,20.323761,181480600,0.016369,1.64,2
2022-07-29,"Petrobras (PETR4) tem preço-alvo elevado, com ...","[Petrobras (PETR4), Especialistas, Petrobras (...",https://www.suno.com.br/noticias/petrobras-pet...,14:21:00,33.599998,34.660000,32.830002,34.150002,20.323761,181480600,0.016369,1.64,2


In [15]:
df_overview_categories = pd.DataFrame(df_raw.value_counts('label').sort_index())
df_overview_categories = df_overview_categories.assign(category_name=['Desceu', 'Constante', 'Subiu'])
display(df_overview_categories)

,0,category_name
label,,
0,2113,Desceu
1,435,Constante
2,1883,Subiu


## Defining train, val and test

In [16]:
from sklearn.model_selection import train_test_split

# Split data into train and test
train, rem = train_test_split(df_raw, train_size=0.8, random_state=52)  # 80% train

val, test = train_test_split(rem, test_size=0.5, random_state=52)  # 10% val, 10% test

print(train.value_counts('label').sort_index())
print(val.value_counts('label').sort_index())
print(test.value_counts('label').sort_index())
print("Total treino: ", train.shape[0])
print("Total validação: ", val.shape[0])
print("Total teste: ", test.shape[0])

label
0    1678
1     354
2    1512
dtype: int64
label
0    221
1     46
2    176
dtype: int64
label
0    214
1     35
2    195
dtype: int64
Total treino:  3544
Total validação:  443
Total teste:  444


## Export full df, train, val and test for future use

In [17]:
train.to_csv('../assets/data/splits/train/raw.csv', index=False)
val.to_csv('../assets/data/splits/val/raw.csv', index=False)
test.to_csv('../assets/data/splits/test/raw.csv', index=False)
df_raw.to_csv('../assets/data/splits/full/raw.csv', index=False)